<a href="https://colab.research.google.com/github/mat10599/QB_hackaton/blob/main/QB_challenge_aristide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import scipy.optimize
import scipy
import pandas as pd
from tensorflow import keras
import os
import random
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import EfficientNetB0



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Load the data
df = pd.read_csv('drive/MyDrive/data/train_data/metadata.csv')
df["path"] = "drive/MyDrive/data/train_data/" + df["path"] + ".tif"
df.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,drive/MyDrive/data/train_data/images/plume/202...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,drive/MyDrive/data/train_data/images/plume/202...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,drive/MyDrive/data/train_data/images/plume/202...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,drive/MyDrive/data/train_data/images/plume/202...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,drive/MyDrive/data/train_data/images/plume/202...


In [8]:
# Split the data
train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)

## CNN de Base

### preprocess

In [130]:
# Split the data
image_height = 224
image_width = 224
batch_size = 32
num_epochs = 15


train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = []
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

# Create an instance of the ImageDataGenerator
traindatagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # preprocessing_function=upscale_image,
)

valdatagen = ImageDataGenerator(
)

train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = traindatagen.flow(train_images, train_labels, batch_size=batch_size)
validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = valdatagen.flow(validation_images, validation_labels, batch_size=batch_size)
test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = valdatagen.flow(test_images, test_labels, batch_size=batch_size)

### model

In [29]:
# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the feature maps
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# Train the model
epochs = 15  # Adjust the number of training epochs as needed
model.fit(train_datagen, validation_data=validation_datagen, epochs=epochs)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_datagen)
print("Test accuracy:", test_accuracy)

Epoch 1/15
16/16 [==============================] - 43s 3s/step - loss: 0.8047 - auc: 0.4781 - val_loss: 0.7124 - val_auc: 0.7472
Epoch 2/15
16/16 [==============================] - 39s 2s/step - loss: 0.6881 - auc: 0.5669 - val_loss: 0.6895 - val_auc: 0.8610
Epoch 3/15
16/16 [==============================] - 40s 2s/step - loss: 0.6693 - auc: 0.6793 - val_loss: 0.6887 - val_auc: 0.8190
Epoch 4/15
16/16 [==============================] - 41s 3s/step - loss: 0.6712 - auc: 0.6691 - val_loss: 0.7020 - val_auc: 0.8590
Epoch 5/15
16/16 [==============================] - 41s 3s/step - loss: 0.6702 - auc: 0.6840 - val_loss: 0.7154 - val_auc: 0.8303
Epoch 6/15
16/16 [==============================] - 39s 2s/step - loss: 0.6455 - auc: 0.7764 - val_loss: 0.6513 - val_auc: 0.9159
Epoch 7/15
16/16 [==============================] - 39s 2s/step - loss: 0.6345 - auc: 0.7772 - val_loss: 0.6512 - val_auc: 0.8933
Epoch 8/15
16/16 [==============================] - 40s 2s/step - loss: 0.6255 - auc: 0.79

## EfficientNet

In [157]:
# Split the data
image_height = 224
image_width = 224
batch_size = 32
num_epochs = 10


train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = []
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

# Create an instance of the ImageDataGenerator
traindatagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # preprocessing_function=upscale_image,
)

valdatagen = ImageDataGenerator(
)

# train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = traindatagen.flow(train_images, train_labels, batch_size=batch_size)
# validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = valdatagen.flow(validation_images, validation_labels, batch_size=batch_size)
# test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = valdatagen.flow(test_images, test_labels, batch_size=batch_size)

In [161]:
def build_model():
    inputs = layers.Input(shape=(224, 224, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    # x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC"]
    )
    return model

model = build_model()

epochs = 10
hist = model.fit(train_datagen, epochs=epochs, validation_data=validation_datagen, verbose=2)

Epoch 1/10
10/10 - 42s - loss: 0.7014 - auc: 0.4633 - val_loss: 0.6830 - val_auc: 0.5000 - 42s/epoch - 4s/step
Epoch 2/10
10/10 - 31s - loss: 0.7099 - auc: 0.4368 - val_loss: 0.7193 - val_auc: 0.5513 - 31s/epoch - 3s/step
Epoch 3/10
10/10 - 32s - loss: 0.6939 - auc: 0.5028 - val_loss: 0.6909 - val_auc: 0.5344 - 32s/epoch - 3s/step
Epoch 4/10
10/10 - 34s - loss: 0.6948 - auc: 0.5025 - val_loss: 0.7012 - val_auc: 0.5262 - 34s/epoch - 3s/step
Epoch 5/10
10/10 - 32s - loss: 0.6974 - auc: 0.4824 - val_loss: 0.7085 - val_auc: 0.4831 - 32s/epoch - 3s/step
Epoch 6/10
10/10 - 30s - loss: 0.6921 - auc: 0.5190 - val_loss: 0.6982 - val_auc: 0.4903 - 30s/epoch - 3s/step
Epoch 7/10
10/10 - 32s - loss: 0.6993 - auc: 0.4739 - val_loss: 0.7237 - val_auc: 0.5200 - 32s/epoch - 3s/step
Epoch 8/10


KeyboardInterrupt: ignored

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
model.save('EfficientNet1.h5')

## VGG

In [ ]:
# Split the data
image_height = 224
image_width = 224
batch_size = 32
num_epochs = 10


train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = []
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

# Create an instance of the ImageDataGenerator
traindatagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # preprocessing_function=upscale_image,
)

valdatagen = ImageDataGenerator(
)

# train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = traindatagen.flow(train_images, train_labels, batch_size=batch_size)
# validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = valdatagen.flow(validation_images, validation_labels, batch_size=batch_size)
# test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = valdatagen.flow(test_images, test_labels, batch_size=batch_size)

In [134]:
# Load the pre-trained EfficientNetB0 model
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

history = model.fit(train_datagen,
                    validation_data=validation_datagen,
                    epochs=num_epochs)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)

Epoch 1/10
10/10 [==============================] - 240s 24s/step - loss: 2.5856 - auc: 0.5409 - val_loss: 3.1383 - val_auc: 0.6149
Epoch 2/10
10/10 [==============================] - 238s 24s/step - loss: 1.1069 - auc: 0.6702 - val_loss: 0.7945 - val_auc: 0.7579
Epoch 3/10
10/10 [==============================] - 281s 29s/step - loss: 0.7921 - auc: 0.7197 - val_loss: 0.6496 - val_auc: 0.7882
Epoch 4/10
10/10 [==============================] - 277s 28s/step - loss: 0.6656 - auc: 0.7652 - val_loss: 0.5135 - val_auc: 0.8210
Epoch 5/10
10/10 [==============================] - 237s 24s/step - loss: 0.7075 - auc: 0.7305 - val_loss: 1.1844 - val_auc: 0.8426
Epoch 6/10
10/10 [==============================] - 279s 29s/step - loss: 0.8177 - auc: 0.6892 - val_loss: 0.7768 - val_auc: 0.8354
Epoch 7/10
10/10 [==============================] - 286s 29s/step - loss: 0.5526 - auc: 0.8141 - val_loss: 0.5396 - val_auc: 0.8369
Epoch 8/10
10/10 [==============================] - 298s 31s/step - loss: 0.

In [146]:
model.save("VGG1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## MobileNet


### preprocess

In [162]:
# Split the data
image_height = 224
image_width = 224
batch_size = 32
num_epochs = 10


train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = []
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

# Create an instance of the ImageDataGenerator
traindatagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # preprocessing_function=upscale_image,
)

valdatagen = ImageDataGenerator(
)

# train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = traindatagen.flow(train_images, train_labels, batch_size=batch_size)
# validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = valdatagen.flow(validation_images, validation_labels, batch_size=batch_size)
# test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = valdatagen.flow(test_images, test_labels, batch_size=batch_size)

### model

In [165]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import binary_crossentropy

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# predictions = Dense(1, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)


model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=0.001), loss=binary_crossentropy, metrics=['AUC'])

# for layer in base_model.layers:
#     layer.trainable = True

# # It's good to use a smaller learning rate for fine-tuning

# model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc',
                                                 mode = 'max',
                                                 min_delta = 0.005,
                                                 patience = 10,
                                                 verbose = 1,
                                                 restore_best_weights = True)

model.fit(train_datagen, epochs=20, validation_data=validation_datagen, callbacks=[early_stopper])

Epoch 1/20
10/10 [==============================] - 26s 2s/step - loss: 0.9443 - auc: 0.4761 - val_loss: 0.6443 - val_auc: 0.6938
Epoch 2/20
10/10 [==============================] - 25s 2s/step - loss: 0.6364 - auc: 0.6758 - val_loss: 0.6688 - val_auc: 0.7728
Epoch 3/20
10/10 [==============================] - 17s 2s/step - loss: 0.6634 - auc: 0.6609 - val_loss: 0.5748 - val_auc: 0.7482
Epoch 4/20
10/10 [==============================] - 30s 3s/step - loss: 0.5835 - auc: 0.7592 - val_loss: 0.6173 - val_auc: 0.7826
Epoch 5/20
10/10 [==============================] - 26s 3s/step - loss: 0.5695 - auc: 0.7752 - val_loss: 0.7672 - val_auc: 0.7764
Epoch 6/20
10/10 [==============================] - 19s 2s/step - loss: 0.6174 - auc: 0.7239 - val_loss: 0.6392 - val_auc: 0.7651
Epoch 7/20
10/10 [==============================] - 20s 2s/step - loss: 0.6079 - auc: 0.7342 - val_loss: 0.5683 - val_auc: 0.7462
Epoch 8/20
10/10 [==============================] - 18s 2s/step - loss: 0.6062 - auc: 0.74

In [166]:
# model.save('MobileNet1.h5')
test_loss, test_accuracy = model.evaluate(test_datagen)
print(f"Test accuracy: {test_accuracy:.2f}")

3/3 [==============================] - 2s 529ms/step - loss: 0.5328 - auc: 0.8046
Test accuracy: 0.80


In [140]:
sample_batch, sample_labels = next(validation_datagen)
sample_batch, sample_labels = next(train_datagen)

data_type = sample_batch.dtype

# Print the data type
print("Data type of the batch:", data_type)

Data type of the batch: float32


## RESNET




In [141]:
train_labels.dtype

dtype('int64')

In [167]:
# Split the data
image_height = 224
image_width = 224
batch_size = 32
num_epochs = 10


train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = []
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.convert('RGB')  # Convert the image to RGB format (3 channels)
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

# Create an instance of the ImageDataGenerator
traindatagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # preprocessing_function=upscale_image,
)

valdatagen = ImageDataGenerator(
)

# train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = traindatagen.flow(train_images, train_labels, batch_size=batch_size)
# validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = valdatagen.flow(validation_images, validation_labels, batch_size=batch_size)
# test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = valdatagen.flow(test_images, test_labels, batch_size=batch_size)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output

# Apply GlobalAveragePooling2D layer once
x = GlobalAveragePooling2D()(x)

# Add your fully connected layers
# x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# Create the output layer for binary classification (1 neuron, sigmoid activation)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001), loss=binary_crossentropy, metrics=['AUC'])

# for layer in base_model.layers:
#     layer.trainable = True

# # It's good to use a smaller learning rate for fine-tuning

# model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc',
                                                 mode = 'max',
                                                 min_delta = 0.005,
                                                 patience = 10,
                                                 verbose = 1,
                                                 restore_best_weights = True)

model.fit(train_datagen, epochs=20, validation_data=validation_datagen, callbacks=[early_stopper])



Epoch 1/20


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7d89d9facb80>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


10/10 [==============================] - 192s 9s/step - loss: 1.0047 - auc: 0.5456 - val_loss: 0.7121 - val_auc: 0.7000
Epoch 2/20
10/10 [==============================] - 88s 9s/step - loss: 0.7351 - auc: 0.5509 - val_loss: 0.8295 - val_auc: 0.6954
Epoch 3/20
10/10 [==============================] - 87s 9s/step - loss: 0.6948 - auc: 0.5497 - val_loss: 0.7262 - val_auc: 0.6744
Epoch 4/20
 5/10 [==============>...............] - ETA: 28s - loss: 0.7153 - auc: 0.5679

In [170]:
train_images.shape

(301, 224, 224, 3)